In [142]:
import influxdb_client
from influxdb_client import InfluxDBClient, Point, WriteOptions
from csv import DictReader
from influxdb_client.client.write_api import SYNCHRONOUS
from tqdm import tqdm
import datetime as dt
import pytz
import pandas as pd
import numpy as np

In [143]:
bucket = "reseau2"
org = "rl-core"
token = "iIQxLS4Gdc1onQX5c4G_aTDYNoU8XoknoVpK1o0s5D3wbeWg_YIPjHYipLhDb2KLI3MsajYbVDOqXVKLlPwrXQ=="
url="http://localhost:8086"

In [144]:
client = influxdb_client.InfluxDBClient(
   url=url,
   token=token,
   org=org
)

write_client = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

In [145]:
def date_to_timestamp(date):
    """
    Input: date string 
    Returns: timestamp for that date
    """
    
    # add leading 0 to hour
    time_list = date.split(' ')
    if time_list[1][1] == ':':
        time_list[1] = "0" + time_list[1]
    time_str = " ".join(time_list)
    
    formats = ['%Y-%m-%d %I:%M:%S %p', '%d/%m/%Y %I:%M:%S %p'] #valid formats in the dataset
    valid_format = False
  
    for format in formats:
        try:
            time = dt.datetime.strptime(time_str, format)
            valid_format   = True
            break
        except:
            pass
    assert valid_format
   
    return dt.datetime.timestamp(time)
     
        

In [146]:
def parse_row(row, date_col, data_cols, col_names):
    """
    Parse row of CSV file into Point 
    
    args:
        row: dict representing one row of csv
        date_col: label of date within the dict
        data_cols: labels of data (i.e. non-date) columns in the dictionary
        col_names: columns to rename data_cols to
    """
    
    time = date_to_timestamp(row[date_col])
    time -= 7 * 60*60 # to adjust for the conversion between GMT and MST     
    
    point = Point("reading")
    for i in range(len(data_cols)):
        point = point.field(col_names[i], float(row[data_cols[i]])) 
        
    point = point.time(int(time*1e9)) # multiplication to convert timestamp in seconds to nanoseconds
    return point


In [147]:
date_col = "Date "
data_cols = [
        " AIT301 ppm ",
        " AIT101 NTU ",
        " AIT401 NTU ", 
        " FIT101 Lpm ", 
        " FIT210 Lpm ", 
        " FIT230 Lpm ", 
        " FIT250 Lpm ", 
        " FIT401 Lpm ", 
        " PT100 psi ", 
        " PT101 psi ", 
        " PT151 psi ", 
        " PT161 psi ", 
        " PT171 psi ", 
        " PT301 psi ", 
        " PT302 psi"]

col_names = [
        "AIT301 ppm",
        "AIT101 NTU",
        "AIT401 NTU", 
        "FIT101 Lpm", 
        "FIT210 Lpm", 
        "FIT230 Lpm", 
        "FIT250 Lpm", 
        "FIT401 Lpm", 
        "PT100 psi", 
        "PT101 psi", 
        "PT151 psi", 
        "PT161 psi", 
        "PT171 psi", 
        "PT301 psi", 
        "PT302 psi"]

In [148]:
# write data 
pth = '/Users/revanmacqueen/RL-Core/root/control/src/environment/reseau/datalog_totals_2023_11/datalog_2023_12.csv'

dataset = DictReader(open(pth, 'r'))
record = []

for i, datum in tqdm(enumerate(dataset)):
    point = parse_row(datum, date_col, data_cols, col_names)
    record.append(point)
   
    if i >= 100:
        break

write_client.write("reseau2", "rl-core", record)

100it [00:00, 80644.18it/s]


In [149]:
def query(query_api, col_names, start_time, end_time):
    """
    """
    
    col_names_str = str(col_names).replace("\'", "\"")
    query_str_list = [
        'from(bucket:"reseau2") ',
        '|> range(start: {}, stop: {}) '.format(start_time, end_time),
        '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") ',
        '|> keep(columns: {})'.format(col_names_str)
        ]
    query_str = ' '.join(query_str_list)
    return query_api.query_data_frame(query_str)
    

In [150]:
query(query_api, col_names, 1701687121, 1701687241)

,result,table,AIT101 NTU,AIT301 ppm,AIT401 NTU,FIT101 Lpm,FIT210 Lpm,FIT230 Lpm,FIT250 Lpm,FIT401 Lpm,PT100 psi,PT101 psi,PT151 psi,PT161 psi,PT171 psi,PT301 psi,PT302 psi
0,_result,0,9.972511,0.0,0.0,-0.036169,-0.014468,-1.262298,-0.018084,0.057870,0.417752,0.461516,0.0,0.456272,0.162580,0.0,0.423177
1,_result,0,9.973597,0.0,0.0,-0.032552,-0.018084,-1.247830,-0.028935,0.061487,0.406901,0.461516,0.0,0.456272,0.136357,0.0,0.439453
